In [1]:
!apt install tesseract-ocr -y
!pip install pytesseract pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
import pytesseract
from PIL import Image
from google.colab import files
import json
import os

# Upload your image
uploaded = files.upload()  # choose your medical report image (e.g., report.jpg)

# Load the uploaded image
image_path = list(uploaded.keys())[0]
img = Image.open(image_path)

# OCR extraction
extracted_text = pytesseract.image_to_string(img).replace("\n", " ").strip()

# Convert directly to JSON schema your chatbot uses
ocr_entry = {
    "question": "What does the uploaded medical report contain?",
    "answers": [extracted_text],
    "context": "OCR extracted text from medical image"
}

# Create the 'knowledge' directory if it doesn't exist
if not os.path.exists("knowledge"):
    os.makedirs("knowledge")

# Save JSON
with open("knowledge/ocr_report.json", "w") as f:
    json.dump([ocr_entry], f, indent=4)

print("✅ OCR text saved directly as JSON in knowledge/ocr_report.json")
print(json.dumps(ocr_entry, indent=4))

Saving pp-plus-activity-report-1024x880-768x660.png to pp-plus-activity-report-1024x880-768x660.png
✅ OCR text saved directly as JSON in knowledge/ocr_report.json
{
    "question": "What does the uploaded medical report contain?",
    "answers": [
        "Physiopedia Plus Activity Report  Name: John Smith  Joined PP+: 25th Aug 2015  Report generated: ath Feb 2016  \u2018Activity for period: 1st Jan 2016 - 9th Feb 2016 (39 days)     Highlights Total PP+ points awarded: 20.1  PP+ Leam topics completed: 1 = Tendinopathy  PP+ Badges awarded: 2 ~75 Physiopedia pages completely read = 50 journal articles completely read  Gon Ww              Activity log Number of activities logged during this period: 46  Date Typ0 Detas Topic Engagement '3170172016 [Embedded video | interview with Alison Hoens-tendinopathy took. Tendinopathy Viewed completly [3170172016 | Jourral arte [Physiotherapy management oflateral epicondyaigia | Tendinopathy Read completly 3170372016 | Journal arid [A reatment algori

In [ ]:
# Add OCR report into chunks
with open("report.txt", "r") as f:
    ocr_text = f.read()

chunks.append(f"OCR Report: {ocr_text}")


In [ ]:
# Add OCR report into chunks
with open("report.txt", "r") as f:
    ocr_text = f.read()

chunks = []  # Initialize chunks as an empty list
chunks.append(f"OCR Report: {ocr_text}")

display(chunks)

['OCR Report: Gem PHYSIOTHERAPY CASE HISTORY FORM     To help us prepare for your TELEHEALTH appointment, we need some information about how you are currently feeling and managing your health condition, as well as information about your history. This information will help us get a thorough understanding of your medical history and presenting complaint and the difficulties you are experiencing. It will also allow us to provide the most accurate assessment and treatment during your TELEHEALTH appointment.     PLEASE COMPLETE THE FOLLOWING QUESTIONS AS FULLY AND ACCURATELY AS.                                      POSSIBLE PERSONAL DETAILS Name: Date of Birth: Date: ‘Sex: Male [ ] Female[ ] Other] | Parent/Guardian Name (if applicable): Phone Number (home): Phone Number (mobile): Address: Email: Primary LanguageSpoken: (Other Languages Spoken: Please Tick if you identify as any of the following: Aboriginal Torres Strait Islander Refugee Non-English-speaking background                      

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.
In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

Before you can make any API calls, you need to initialize the Generative Model.

In [ ]:
# Initialize the Gemini API
gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-04-17')

Now, you can use the extracted text from the report as context and ask a question using the model:

In [ ]:
import json

file_path = "knowledge/ocr_report.json"

# Load the JSON data from the file
with open(file_path, "r") as f:
    loaded_data = json.load(f)

# Now 'loaded_data' contains the content of your JSON file
# You can print it to see its structure and content
print(loaded_data)

[{'question': 'What does the uploaded medical report contain?', 'answers': ["Physiopedia Plus Activity Report  Name: John Smith  Joined PP+: 25th Aug 2015  Report generated: ath Feb 2016  ‘Activity for period: 1st Jan 2016 - 9th Feb 2016 (39 days)     Highlights Total PP+ points awarded: 20.1  PP+ Leam topics completed: 1 = Tendinopathy  PP+ Badges awarded: 2 ~75 Physiopedia pages completely read = 50 journal articles completely read  Gon Ww              Activity log Number of activities logged during this period: 46  Date Typ0 Detas Topic Engagement '3170172016 [Embedded video | interview with Alison Hoens-tendinopathy took. Tendinopathy Viewed completly [3170172016 | Jourral arte [Physiotherapy management oflateral epicondyaigia | Tendinopathy Read completly 3170372016 | Journal arid [A reatment algorithm fr managing Achiles tondin Tendinopathy Read completely"], 'context': 'OCR extracted text from medical image'}]
